### 不同模式全球变化斜率  saved as nc file

In [1]:
from netCDF4 import Dataset#可以拼接数据集
import numpy as np
import pandas as pd
import netCDF4
import matplotlib.pyplot as plt
from osgeo import gdal
import os
name_dir = ['CNRM-CM5','GFDL-CM3', 'GISS-E2-R', 'ACCESS1-0','MPI-ESM-LR'] #
'''五种模式行列数分别为 温度单位为K， 转换成摄氏度需要-273.15 ， 降雨单位 kg m-2 s-1 折算成mm 需要*24*3600
(1140, 128, 256)  a = [i for i in range(0,30,5)]
(1140, 90, 144) b = [i for i in range(1,30,5)]
(1140, 90, 144)
(1140, 145, 192)
(1140, 96, 192)
'''
name = []
path =  [r'I:\future_clim\pr\4.5',r'I:\future_clim\pr\8.5',r'I:\future_clim\tasmax\4.5',r'I:\future_clim\tasmax\8.5',
         r'I:\future_clim\tasmin\4.5',r'I:\future_clim\tasmin\8.5']
dirs = []
for i in path:
    for j in name_dir:
        dirs.append(i + os.sep + j)
var = ['pr', 'pr', 'tasmax', 'tasmax', 'tasmin', 'tasmin']


### Linear regression function

In [24]:
import numba as nb
from sklearn import datasets, linear_model

year = [i for i in range(2006,2101,1)]
year = np.array(year).reshape(95,1)# year = 95

@nb.jit()
def slope_(arr):
    regr = linear_model.LinearRegression()
    regr.fit(year,arr) 
    return regr.coef_

In [27]:
#write tif
def writeTiff(im_data,im_width,im_height,im_bands,im_geotrans,im_proj,path):
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    else:
        datatype = gdal.GDT_Float32

    if len(im_data.shape) == 3:
        im_bands, im_height, im_width = im_data.shape
    elif len(im_data.shape) == 2:
        im_data = np.array([im_data])
    else:
        im_bands, (im_height, im_width) = 1,im_data.shape
        #创建文件
    driver = gdal.GetDriverByName("GTiff")
    dataset = driver.Create(path, im_width, im_height, im_bands, datatype)
    if(dataset!= None):
        dataset.SetGeoTransform(im_geotrans) #写入仿射变换参数
        dataset.SetProjection(im_proj) #写入投影
    for i in range(im_bands):
        dataset.GetRasterBand(i+1).WriteArray(im_data[i])
    del dataset

In [28]:
from osgeo import gdal
ds = gdal.Open(r'H:\china_geodata\China_soil_1km\ChinaSoil_1km.tif')
prj = ds.GetProjection()
#tr = ds.GetGeoTransform()
tr = (0,1.4, 0.0, 90, 0.0, -1.4)

a = [i for i in range(0,30,5)]#'CNRM-CM5',(1140, 128, 256)      1.4   1.4
b = [i for i in range(1,30,5)]#'GFDL-CM3',(1140, 90, 144)        2    2.5
c = [i for i in range(2,30,5)]# 'GISS-E2-R', (1140, 90, 144)     2    2.5
d = [i for i in range(3,30,5)]#'ACCESS1-0',(1140, 145, 192)     1.25  1.875
e = [i for i in range(4,30,5)]#'MPI-ESM-LR'(1140, 96, 192)      1.86  1.875


In [ ]:
for i in a:#len(dirs)=30
    data = netCDF4.MFDataset([(dirs[i]+ os.sep+j) for j in os.listdir(dirs[i]) if  j.endswith('nc')])
    if i <10:
        model = 4.5
        for mon in range(12):
            slope = []
            for id_x in range(128):
                for id_y in range(256):
                    slope.append(slope_(data['pr'][mon::12, id_x,id_y]*24*3600*30))
            writeTiff(np.array(slope).reshape(128,256),256, 128, 1,tr, prj,r'I:\future_clim\slope'+'//'+'CNRM_pr%f_%d.tif'%(model, mon+1))
        model = model + 4
    elif i >=10 and i <20:
        model = 4.5
        for mon in range(12):
            slope = []
            for id_x in range(128):
                for id_y in range(256):
                    slope.append(slope_(data['tasmax'][mon::12, id_x,id_y]-273.15))
            writeTiff(np.array(slope).reshape(128,256),256, 128, 1,tr, prj,r'I:\future_clim\slope'+'//'+'CNRM_tasmax%f_%d.tif'%(model, mon+1))
        model = model + 4
    else:
        model = 4.5
        for mon in range(12):
            slope = []
            for id_x in range(128):
                for id_y in range(256):
                    slope.append(slope_(data['pr'][mon::12, id_x,id_y]-273.15))
            writeTiff(np.array(slope).reshape(128,256),256, 128, 1,tr, prj,r'I:\future_clim\slope'+'//'+'CNRM_tasmin%f_%d.tif'%(model, mon+1))
        model = model + 4
 ###########################       

In [26]:
%time slope_(data['tasmin'][0::12,1,1])

Wall time: 8.98 ms


array([0.0492186])